Baseline model either KNN or svm
Look at how it is done by Yeh Chris etc.

In [ ]:
print('yooyo')

In [1]:
#!pip install --upgrade keras-cv tensorflow
#!pip install --upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.4 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1


In [12]:
import tensorflow as tf
from tensorflow import keras
import keras_cv
from keras_cv import layers as cv_layers
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import pickle
import glob
import cv2
import re
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn import metrics


print("All imports loaded successfully!")


All imports loaded successfully!


In [3]:
print(tf.__version__)
print(keras.__version__)
print(keras_cv.__version__)

2.17.0
3.6.0
0.9.0


In [ ]:
print('All run ready for datas')

In [4]:
# Connect to Drive folder with Data
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/My Drive/Sarah_Master_Project/Data/Colab_data'

#TODO: this is for Colab obviously change this if in vscode

Mounted at /content/drive


In [5]:
image_file = os.path.join(folder_path, 'Kenya2019_array_augmented.npy')
images = np.load(image_file, allow_pickle=True)
images = images.item()


In [6]:

df = pd.read_csv(os.path.join(folder_path, 'clustered_geopoints.txt'), sep = '\t')
labels = df['avg_wealthscore'].values
image_codes = df['image_code'].astype(str).values

#Remove nan
non_nan = ~np.isnan(labels)
labels = labels[non_nan]
image_codes = image_codes[non_nan]

In [7]:
df['avg_wealthscore'].value_counts()

,count
avg_wealthscore,
3.0,517
2.0,461
1.0,418
4.0,413
5.0,357
2.5,23
4.5,15
3.5,13
1.5,10


In [8]:
X = np.array([images[code] for code in image_codes if code in images])


print(f"Image data shape: {X.shape}")
print(f"Labels shape: {labels.shape}")

Image data shape: (2227, 224, 224, 3)
Labels shape: (2227,)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)


# Flat arrays
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)


In [11]:
knn_regressor = KNeighborsRegressor()

param_grid = {'n_neighbors': np.arange(1, 25)}
knn_gscv = GridSearchCV(knn_regressor, param_grid, cv=5)

# Train
knn_gscv.fit(X_train_flat, y_train)

# Predict
#y_pred = knn_regressor.predict(X_test_flat)
y_pred = knn_gscv.predict(X_test_flat)



# Evaluate
mse = mean_squared_error(y_test, y_pred)
print(f"Baseline KNN Regressor Mean Squared Error: {mse}")

Baseline KNN Regressor Mean Squared Error: 1.5358121574489287


In [ ]:
# Grid search: parameters from kaggle reference
grid_params = { 'n_neighbors' : [5,7,9,11,13,15],
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}


# Add paramaeters to search
knn_grid = GridSearchCV(knn_regressor, grid_params, verbose = 1, cv=5, n_jobs = -1)

# Fit the model
knn_grid_model= knn_grid.fit(X_train_flat, y_train)



Fitting 5 folds for each of 36 candidates, totalling 180 fits


In [ ]:
print(knn_grid_model.best_score_)
print(knn_grid_model.best_params_)



In [ ]:
# Based off best results
knn = knn_regressor(n_neighbors = 5, weights = 'uniform',algorithm = 'brute',metric = 'minkowski')
knn.fit(X_train_flat, y_train)


In [ ]:
y_pred = knn.predict(X_test_flat)
y_hat = knn.predict(X_train_flat)
mse = mean_squared_error(y_test, y_pred)
print(f"KNN Regressor Mean Squared Error: {mse}")

In [ ]:
print('Training set accuracy: ', metrics.accuracy_score(y_train, y_hat))
print('Test set accuracy: ',metrics.accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(knn, X, y, cv =5)
print('Model accuracy: ',np.mean(scores))


Good KNN references:


https://www.kaggle.com/code/arunimsamudra/k-nn-with-hyperparameter-tuning


https://medium.com/@madhuri15/significance-of-k-nearest-neighbors-in-machine-learning-572f0daa2841